In [ ]:
import numpy as np
import overpy

api = overpy.Overpass()

query = """
[out:json];
(
  way["building"](52.5100,13.3950,52.5400,13.4250);
);
out body;
>;
out skel qt;
"""

result = api.query(query)

buildings_data = []

i = 0
for way in result.ways:
    if i >= 1000:
        break
    nodes_data = []
    
    for node in way.nodes:
        lat, lon = node.lat, node.lon
        nodes_data.append([lat, lon])
    
    nodes_array = np.array(nodes_data)
    
    buildings_data.append(nodes_array)
    i += 1


In [ ]:
import numpy as np
from shapely.geometry import Polygon
from pyproj import Proj, transform

proj_utm = Proj(proj='utm', zone=33, ellps='WGS84')

areas = []
perimeters = []

for i, building in enumerate(buildings_data):
    utm_coords = []
    for lat, lon in building:
        x, y = proj_utm(lon, lat)
        utm_coords.append((x, y))
    
    polygon = Polygon(utm_coords)
    
    area = polygon.area
    perimeter = polygon.length

    areas.append(area)
    perimeters.append(perimeter)


In [ ]:
import csv

csv_data = []


for i, (area, perimeter) in enumerate(zip(areas, perimeters)):

    csv_data.append([i + 1, area, perimeter])


with open('building_metrics.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(['Building Index', 'Area (sqm)', 'Perimeter (m)'])
    
    writer.writerows(csv_data)



下面的暂时没用

In [4]:
import xml.etree.ElementTree as ET

def save_osm(result, file_name="buildings.osm"):
    osm_root = ET.Element("osm", version="0.6")

    # 添加节点
    for node in result.nodes:
        ET.SubElement(osm_root, "node", id=str(node.id), lat=str(node.lat), lon=str(node.lon))

    # 添加建筑物路径（多边形）
    for way in result.ways:
        way_element = ET.SubElement(osm_root, "way", id=str(way.id))
        for node in way.nodes:
            ET.SubElement(way_element, "nd", ref=str(node.id))

    # 保存为 .osm 文件
    tree = ET.ElementTree(osm_root)
    tree.write(file_name)

# 保存结果到buildings.osm文件
save_osm(result)

In [10]:
import subprocess

def convert_to_3d(input_file="buildings.osm", output_file="buildings.obj"):
    command = [
        "java", "-jar", "/Users/alserial/Downloads/OSM2World-0.3.1-bin/OSM2World.jar",  # osm2world的路径
        "-i", input_file,  # 输入的 OSM 文件
        "-o", output_file  # 输出的 OBJ 文件
    ]

    # 执行命令
    result = subprocess.run(command, capture_output=True, text=True)

    # 检查是否有错误
    if result.returncode == 0:
        print(f"Conversion successful, 3D model saved as {output_file}")
    else:
        print(f"Error during conversion: {result.stderr}")

# 调用 osm2world 进行三维转换
convert_to_3d("buildings.osm", "buildings.obj")

Conversion successful, 3D model saved as buildings.obj
